# Functional model

In [1]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam
from keras import layers
from keras.models import Model
from keras.utils import plot_model

#load the MNIST dataset from keras datasets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#Process data
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1) # Expend dimension for 1 cahnnel image
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)  # Expend dimension for 1 cahnnel image
X_train = X_train / 255 # Normalize
X_test = X_test / 255 # Normalize

#One hot encoding
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_test = np_utils.to_categorical(y_test, num_classes=10)

# Build LetNet model using sequential model
def LetNet(width, height, depth, classes):
    # initialize the model
    model = Sequential()

    # first layer, convolution and pooling
    model.add(Conv2D(input_shape=(width, height, depth), kernel_size=(5, 5), filters=6, strides=(1,1), activation='tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # second layer, convolution and pooling
    model.add(Conv2D(input_shape=(width, height, depth), kernel_size=(5, 5), filters=16, strides=(1,1), activation='tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Fully connection layer
    model.add(Flatten())
    model.add(Dense(120,activation = 'tanh'))
    model.add(Dense(84,activation = 'tanh'))

    # softmax classifier
    model.add(Dense(classes))
    model.add(Activation("softmax"))
    return model

# Build LetNet model using functional model
def LetNetFunctional(width, height, depth, classes):
    inputs = layers.Input(shape=(width, height, depth))
    x = layers.Conv2D(kernel_size=(5, 5), filters=6, strides=(1,1), activation='tanh')(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2))(x)

    x = layers.Conv2D(kernel_size=(5, 5), filters=16, strides=(1,1), activation='tanh')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2))(x)

    x = Flatten()(x)
    x = layers.Dense(120, activation='tanh')(x)
    x = layers.Dense(84, activation='tanh')(x)

    outputs = layers.Dense(classes,activation='softmax')(x)
    return Model(inputs ,outputs)

# Build ResNet, multiple input
def ResNetTiny(width, height, depth, classes):
    inputs = keras.Input(shape=(width, height, depth), name="img")
    x = layers.Conv2D(32, 3, activation="relu")(inputs)
    x = layers.Conv2D(64, 3, activation="relu")(x)
    block_1_output = layers.MaxPooling2D(3)(x)

    x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_1_output)
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    # add輸入的維度需要一樣 ex. 8, 8, 64
    block_2_output = layers.add([x, block_1_output])

    # 後在新增兩層卷積並在block_3_output做融合
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_2_output)
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    block_3_output = layers.add([x, block_2_output])

    x = layers.Conv2D(64, 3, activation="relu")(block_3_output)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(classes)(x)

    model = keras.Model(inputs, outputs, name="tiny_resnet")
    return model


# create model
model = ResNetTiny(28,28,1,10)
model.summary()
model.compile(optimizer=Adam(lr=0.001),loss = 'categorical_crossentropy',metrics=['accuracy'])

plot_model(model)

#Strat training
History = model.fit(X_train, y_train, epochs=2, batch_size=32,validation_data=(X_test, y_test))

11493376/11490434 [==============================] - 0s 0us/step
Model: "tiny_resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 26, 26, 32)   320         img[0][0]                        
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 24, 24, 64)   18496       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 8, 8, 64)     0           conv2d_1[0][0]                   
_______________________